<a href="https://www.kaggle.com/code/owaiskhan9654/kaggle-autogen-agent-chat-group-retrieval?scriptVersionId=152183649" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <span style="background-color:black; color:#fda504; padding: 3px;">Kaggle Auto Generated AgentChat Group Chat with Retrieval 🎊</span>

![](https://github.com/Owaiskhan9654/Autogen-Kaggle/blob/main/img/Autogen.jpeg?raw=true)

**Disclaimer - This notebook relies on Azure OpenAI API keys for running the Autogen Model. Kindly use your own keys. I've made multiple attempts to execute Autogen with a local Kaggle GPUs hosted open-source model &#x1F614;, but it's still a work in progress.**

### ***Paper Link*** [AutoGen: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155)

In [1]:
%%capture
!pip install -U ipywidgets
!pip install pyautogen[retrievechat]~=0.1.11

In [2]:
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    azure_api_key_secret = user_secrets.get_secret("azure_api_key_secret")
    api_base_secret = user_secrets.get_secret("api_base_secret")

    anony=None
except:
    anony = "must"
    print("This notebook relies on Azure OpenAI API keys for running the Autogen Model. Kindly use your own keys. I've made multiple attempts to execute Autogen with a locally hosted open-source model :(, but it's still a work in progress.")

In [3]:
config_list = [{
    "engine": "gpt-35-turbo-16k", 
    "model": "gpt-35-turbo-16k",
    "api_base": api_base_secret, # Your Secret Base URL Endpoint :(
    "api_type": "azure", 
    "api_version": "2023-07-01-preview",
    "api_key": azure_api_key_secret # Your Secret Azure API key :(
}]

In [4]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import AssistantAgent
import chromadb
import autogen

llm_config = {
    "request_timeout": 60,
    "seed": 91,
    "config_list": config_list,
    "temperature": 0.3,
}

autogen.ChatCompletion.start_logging()
termination_msg = lambda x: isinstance(x, dict) and "TERMINATE" == str(x.get("content", ""))[-9:].upper()

boss = autogen.UserProxyAgent(
    name="Boss",
    is_termination_msg=termination_msg,
    human_input_mode="NEVER",
    system_message="The boss who ask questions and give tasks.",
    code_execution_config=False,  # we don't want to execute code in this case.
)

boss_aid = RetrieveUserProxyAgent(
    name="Boss_Assistant",
    is_termination_msg=termination_msg,
    system_message="Assistant who has extra content retrieval power for solving difficult problems.",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    retrieve_config={
        "task": "code",
        "docs_path": "https://raw.githubusercontent.com/microsoft/FLAML/main/website/docs/Examples/Integrate%20-%20Spark.md",
        "chunk_token_size": 1200,
        "model": config_list[0]["model"],
        "client": chromadb.PersistentClient(path="owais_experiments/chromadb"),
        "collection_name": "groupchat",
        "get_or_create": True,
    },
    code_execution_config=False,  # we don't want to execute code in this case.
)

owais_coder = AssistantAgent(
    name="Senior_Data_Scientist_Owais",
    is_termination_msg=termination_msg,
    system_message="You are a senior Data Scientist and a Python Engineer. Your Name is Owais. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

pm = autogen.AssistantAgent(
    name="Product_Manager",
    is_termination_msg=termination_msg,
    system_message="You are a product manager. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

reviewer = autogen.AssistantAgent(
    name="Code_Reviewer",
    is_termination_msg=termination_msg,
    system_message="You are a code reviewer. Reply `TERMINATE` in the end when everything is done.",
    llm_config=llm_config,
)

PROBLEM = "How to use spark for parallel training in FLAML? Give me sample code."

def _reset_agents():
    boss.reset()
    boss_aid.reset()
    owais_coder.reset()
    pm.reset()
    reviewer.reset()

## <span style="background-color:black; color:#fda504; padding: 13px;">Now you can see in the output cell how </span>
## <span style="background-color:black; color:#fda504; padding: 13px;">Autogen are interacting with each other in rag_chat and no_rag_chat </span>


In [5]:
def rag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss_aid, owais_coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss_aid as this is the user proxy agent.
    boss_aid.initiate_chat(
        manager,
        problem=PROBLEM,
        n_results=3,)
    
    
rag_chat()

Trying to create collection.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

doc_ids:  [['doc_0', 'doc_1']]
Adding doc_id doc_0 to context.
Adding doc_id doc_1 to context.
Boss_Assistant (to chat_manager):

You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How to use spark for parallel training in FLAML? Give me sample code.

Context is: # Integrate - Spark

FLAML has integrated Spark for distributed training. There are two main aspects of integration with Spark:
- Use Spark ML estimators for AutoML.
- Use Spark to run training in parallel spark jobs.

## Spark ML Estimators

FLAML integrates estimators

In [6]:
def norag_chat():
    _reset_agents()
    groupchat = autogen.GroupChat(
        agents=[boss, owais_coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,)
norag_chat()

Boss (to chat_manager):

How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------
Senior_Data_Scientist_Owais (to chat_manager):

To use Spark for parallel training in FLAML, you need to follow these steps:

1. Install FLAML and PySpark:

```python
!pip install flaml
!pip install pyspark
```

2. Import the necessary libraries:

```python
from flaml import AutoML
from pyspark.sql import SparkSession
```

3. Create a SparkSession:

```python
spark = SparkSession.builder \
    .appName("FLAML Example") \
    .getOrCreate()
```

4. Load your data into a Spark DataFrame:

```python
data = spark.read.format("csv").option("header", "true").load("path/to/your/data.csv")
```

5. Specify the target column and the search space for FLAML:

```python
settings = {
    "time_budget": 60,  # total running time in seconds
    "metric": 'accuracy',  # primary metric for optimization
    "task": 'classification

In [7]:
def call_rag_chat():
    _reset_agents()
    # In this case, we will have multiple user proxy agents and we don't initiate the chat
    # with RAG user proxy agent.
    # In order to use RAG user proxy agent, we need to wrap RAG agents in a function and call
    # it from other agents.
    def retrieve_content(message, n_results=3):
        boss_aid.n_results = n_results  # Set the number of results to be retrieved.
        # Check if we need to update the context.
        update_context_case1, update_context_case2 = boss_aid._check_update_context(message)
        if (update_context_case1 or update_context_case2) and boss_aid.update_context:
            boss_aid.problem = message if not hasattr(boss_aid, "problem") else boss_aid.problem
            _, ret_msg = boss_aid._generate_retrieve_user_reply(message)
        else:
            ret_msg = boss_aid.generate_init_message(message, n_results=n_results)
        return ret_msg if ret_msg else message
    
    boss_aid.human_input_mode = "NEVER" # Disable human input for boss_aid since it only retrieves content.
    
    llm_config = {
        "functions": [
            {
                "name": "retrieve_content",
                "description": "retrieve content for code generation and question answering.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "message": {
                            "type": "string",
                            "description": "Refined message which keeps the original meaning and can be used to retrieve content for code generation and question answering.",
                        }
                    },
                    "required": ["message"],
                },
            },
        ],
        "config_list": config_list,
        "request_timeout": 60,
        "seed": 93,
    }

    for agent in [owais_coder, pm, reviewer]:
        # update llm_config for assistant agents.
        agent.llm_config.update(llm_config)

    for agent in [boss, owais_coder, pm, reviewer]:
        # register functions for all agents.
        agent.register_function(
            function_map={
                "retrieve_content": retrieve_content,
            }
        )

    groupchat = autogen.GroupChat(
        agents=[boss, owais_coder, pm, reviewer], messages=[], max_round=12
    )
    manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)

    # Start chatting with boss as this is the user proxy agent.
    boss.initiate_chat(
        manager,
        message=PROBLEM,
    )
call_rag_chat()

Boss (to chat_manager):

How to use spark for parallel training in FLAML? Give me sample code.

--------------------------------------------------------------------------------
Senior_Data_Scientist_Owais (to chat_manager):

***** Suggested function Call: retrieve_content *****
Arguments: 
{
  "message": "How to use spark for parallel training in FLAML? Give me sample code."
}
*****************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION retrieve_content...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

doc_ids:  [['doc_0', 'doc_1']]
Adding doc_id doc_0 to context.
Adding doc_id doc_1 to context.
Code_Reviewer (to chat_manager):

***** Response from calling function "retrieve_content" *****
You're a retrieve augmented coding assistant. You answer user's questions based on your own knowledge and the
context provided by the user.
If you can't answer the question with or without the current context, you should reply exactly `UPDATE CONTEXT`.
For code generation, you must obey the following rules:
Rule 1. You MUST NOT install any packages because all the packages needed are already installed.
Rule 2. You must follow the formats below to write your code:
```language
# your code
```

User's question is: How to use spark for parallel training in FLAML? Give me sample code.

Context is: # Integrate - Spark

FLAML has integrated Spark for distributed training. There are two main aspects of integration with Spark:
- Use Spark ML estimators for AutoML.
- Use Spark to run training in parallel spa